# Credit Card Customer Churn Analysis

## Business Problem
Credit card companies lose substantial revenue when customers churn. Acquiring a new customer is far more expensive than retaining an existing one, and high churn rates directly reduce profitability. Predicting which customers are likely to leave enables the company to target retention offers more effectively, lowering churn and maximizing return on investment.

## Research Questions
- Which customer characteristics are most predictive of churn?
- Can we build a model to identify at-risk customers before they leave?
- What actionable insights can guide retention strategies?

## Objective
Develop a predictive model to identify customers with high churn probability and provide data-driven recommendations for retention efforts. Quantify the cost-benefit of targeting at-risk customers to demonstrate measurable business ROI from predictive analytics.

---

## Stage 1: Data Discovery & Understanding

Let's start by exploring what data we have available and understanding its structure.

In [95]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


In [96]:
# Load the dataset
df = pd.read_csv('../data/credit_card_customers.csv')

print("Dataset loaded successfully!")
print("Shape:", df.shape)

Dataset loaded successfully!
Shape: (10000, 23)


In [97]:
# First look at the data
print("First 5 rows:")
df.head()

First 5 rows:


,customer_id,gender,age,education_level,marital_status,dependents,income,credit_limit,tenure_months,relationship_duration_years,avg_monthly_spend,monthly_balance,credit_utilization,transaction_count,cash_advance_count,card_category,late_payment_count,missed_payment_flag,last_payment_amount,online_services_used,reward_points,risk_score,churn
0,CUST00001,Male,27,Associate,Married,2,45914.0,13900.0,34,2.0,630.0,3186.0,0.2292,41,1,Business,1,0,2946.0,7,8938.0,300.0,0
1,CUST00002,Female,31,Associate,Married,3,43534.0,8900.0,25,1.4,789.0,2516.0,0.2827,33,3,Standard,1,0,930.0,6,8241.0,311.0,0
2,CUST00003,Female,49,Associate,Married,2,49836.0,7800.0,13,1.1,942.0,2229.0,0.2858,34,1,Business,2,0,1069.0,5,5108.0,316.4,0
3,CUST00004,Female,30,High School,Single,1,28599.0,6700.0,134,10.7,171.0,1738.0,0.2594,36,1,Standard,2,0,885.0,2,9699.0,337.1,0
4,CUST00005,Male,58,High School,Single,0,36715.0,7800.0,34,1.9,635.0,1838.0,0.2356,44,0,Standard,1,0,1282.0,0,9047.0,300.0,0


In [98]:
# Column information
print("Column Information:")
print("="*50)
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

Column Information:
 1. customer_id
 2. gender
 3. age
 4. education_level
 5. marital_status
 6. dependents
 7. income
 8. credit_limit
 9. tenure_months
10. relationship_duration_years
11. avg_monthly_spend
12. monthly_balance
13. credit_utilization
14. transaction_count
15. cash_advance_count
16. card_category
17. late_payment_count
18. missed_payment_flag
19. last_payment_amount
20. online_services_used
21. reward_points
22. risk_score
23. churn


In [99]:
# Data types and basic info
print("Dataset Info:")
df.info()

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   customer_id                  10000 non-null  object 
 1   gender                       10000 non-null  object 
 2   age                          10000 non-null  int64  
 3   education_level              10000 non-null  object 
 4   marital_status               10000 non-null  object 
 5   dependents                   10000 non-null  int64  
 6   income                       9950 non-null   float64
 7   credit_limit                 10000 non-null  float64
 8   tenure_months                10000 non-null  int64  
 9   relationship_duration_years  10000 non-null  float64
 10  avg_monthly_spend            10000 non-null  float64
 11  monthly_balance              10000 non-null  float64
 12  credit_utilization           10000 non-null  float64
 13  tra

In [100]:
# Data Quality Check: Tenure vs Relationship Duration Consistency
print("Data Quality Issue Check:")
print("=" * 50)

# Check if tenure_months and relationship_duration_years are consistent
tenure_months = df['tenure_months']
relationship_years = df['relationship_duration_years']
expected_years = tenure_months / 12

correlation = tenure_months.corr(relationship_years)
print("Tenure vs Relationship Duration Analysis:")
print("tenure_months mean:", tenure_months.mean().round(1))
print("relationship_duration_years mean:", relationship_years.mean().round(1))
print("Expected years from tenure_months:", expected_years.mean().round(1))
print("Correlation between fields:", correlation.round(3))

if correlation < 0.99:
    print("ISSUE IDENTIFIED: tenure_months and relationship_duration_years are inconsistent")
    print("This needs to be fixed in feature engineering stage")
else:
    print("Fields are consistent")

Data Quality Issue Check:
Tenure vs Relationship Duration Analysis:
tenure_months mean: 36.1
relationship_duration_years mean: 2.2
Expected years from tenure_months: 3.0
Correlation between fields: 0.938
ISSUE IDENTIFIED: tenure_months and relationship_duration_years are inconsistent
This needs to be fixed in feature engineering stage


In [ ]:
# Check for our target variable - churn
print("Target Variable Analysis:")
print("="*30)
print("Churn distribution:")
print(df['churn'].value_counts())
print("Churn rate:", str(round(df['churn'].mean()*100, 1)) + "%")

# Visualize churn distribution
plt.figure(figsize=(8, 5))
plt.subplot(1, 2, 1)
df['churn'].value_counts().plot(kind='bar', color=['lightblue', 'lightcoral'])
plt.title('Churn Distribution (Count)')
plt.xlabel('Churn (0=Retained, 1=Churned)')
plt.ylabel('Count')
plt.xticks(rotation=0)

plt.subplot(1, 2, 2)
df['churn'].value_counts(normalize=True).plot(kind='bar', color=['lightblue', 'lightcoral'])
plt.title('Churn Distribution (%)')
plt.xlabel('Churn (0=Retained, 1=Churned)')
plt.ylabel('Percentage')
plt.xticks(rotation=0)

plt.tight_layout()
plt.savefig('../outputs/01_churn_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Key findings
print("Target variable identified: 'churn' (binary classification)")
print("Churn rate of", str(round(df['churn'].mean()*100, 1)) + "% is realistic for credit cards")
print("Dataset has sufficient samples for reliable modeling")

## Stage 2: Data Quality Assessment & Cleaning

Now that we understand the dataset structure, let's assess data quality and identify issues that need to be addressed before modeling.

In [40]:
# Missing values analysis
missing_values = df.isnull().sum()
missing_summary = missing_values[missing_values > 0]
print(missing_summary) if len(missing_summary) > 0 else print("No missing values")

income                 50
last_payment_amount    50
reward_points          50
dtype: int64


In [41]:
# Categorical data inspection
categorical_cols = ['gender', 'education_level', 'marital_status', 'card_category']

for col in categorical_cols:
    print(col + ":", df[col].value_counts().to_dict())

gender: {'Male': 5299, 'Female': 4681, 'M': 10, 'male': 5, 'F': 5}
education_level: {'High School': 3483, 'Associate': 2954, 'Bachelor': 2112, 'Master': 981, 'Doctorate': 455, 'Masters': 5, 'Bachelors': 5, 'PhD': 5}
marital_status: {'Married': 5927, 'Single': 2193, 'Divorced': 1560, 'Widowed': 295, 'married': 8, 'SINGLE': 7, 'widow': 5, 'Div': 5}
card_category: {'Business': 4686, 'Standard': 3887, 'Gold': 1253, 'Platinum': 144, ' Business': 8, ' Standard': 7, 'Business ': 6, 'Standard ': 5, ' Gold': 3, 'Platinum ': 1}


In [42]:
# Data quality checks
print("Negative values:")
print("  avg_monthly_spend:", (df['avg_monthly_spend'] < 0).sum())
print("  last_payment_amount:", (df['last_payment_amount'] < 0).sum())
print("  reward_points:", (df['reward_points'] < 0).sum())

print("Balance > Credit Limit:", (df['monthly_balance'] > df['credit_limit']).sum())

Negative values:
  avg_monthly_spend: 3
  last_payment_amount: 3
  reward_points: 2
Balance > Credit Limit: 2


In [43]:
# Create cleaned dataset
df_clean = df.copy()

# Handle missing values (median imputation for numerical)
df_clean['income'].fillna(df_clean['income'].median(), inplace=True)
df_clean['last_payment_amount'].fillna(df_clean['last_payment_amount'].median(), inplace=True)
df_clean['reward_points'].fillna(df_clean['reward_points'].median(), inplace=True)

# Standardize categorical values
df_clean['gender'] = df_clean['gender'].replace({'M': 'Male', 'male': 'Male', 'F': 'Female'})
df_clean['education_level'] = df_clean['education_level'].replace({'Bachelors': 'Bachelor', 'Masters': 'Master', 'PhD': 'Doctorate'})
df_clean['marital_status'] = df_clean['marital_status'].replace({'married': 'Married', 'SINGLE': 'Single', 'widow': 'Widowed', 'Div': 'Divorced'})
df_clean['card_category'] = df_clean['card_category'].str.strip()  # Remove leading/trailing spaces

# Fix negative values (set to absolute value - assuming data entry errors)
df_clean.loc[df_clean['avg_monthly_spend'] < 0, 'avg_monthly_spend'] = abs(df_clean['avg_monthly_spend'])
df_clean.loc[df_clean['last_payment_amount'] < 0, 'last_payment_amount'] = abs(df_clean['last_payment_amount'])
df_clean.loc[df_clean['reward_points'] < 0, 'reward_points'] = abs(df_clean['reward_points'])

print("Cleaned dataset shape:", df_clean.shape)
print("Missing values remaining:", df_clean.isnull().sum().sum())

Cleaned dataset shape: (10000, 23)
Missing values remaining: 0


In [44]:
# Verify cleaning results
print("Categorical values after cleaning:")
for col in categorical_cols:
    print(col + ":", df_clean[col].unique())

print("Negative values after cleaning:")
print("  avg_monthly_spend:", (df_clean['avg_monthly_spend'] < 0).sum())
print("  last_payment_amount:", (df_clean['last_payment_amount'] < 0).sum())
print("  reward_points:", (df_clean['reward_points'] < 0).sum())

Categorical values after cleaning:
gender: ['Male' 'Female']
education_level: ['Associate' 'High School' 'Bachelor' 'Master' 'Doctorate']
marital_status: ['Married' 'Single' 'Divorced' 'Widowed']
card_category: ['Business' 'Standard' 'Gold' 'Platinum']
Negative values after cleaning:
  avg_monthly_spend: 0
  last_payment_amount: 0
  reward_points: 0


In [45]:
# Save cleaned dataset
df_clean.to_csv('../data/credit_card_customers_cleaned.csv', index=False)
print("Cleaned dataset saved to '../data/credit_card_customers_cleaned.csv'")

Cleaned dataset saved to '../data/credit_card_customers_cleaned.csv'


## Stage 3: Exploratory Data Analysis

Now let's explore customer behavior patterns and identify what drives churn through data visualization and statistical analysis.

In [ ]:
# Demographic analysis by churn
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Age distribution
axes[0,0].hist([df_clean[df_clean['churn']==0]['age'], df_clean[df_clean['churn']==1]['age']], 
               bins=20, alpha=0.7, label=['Retained', 'Churned'], color=['lightblue', 'lightcoral'])
axes[0,0].set_title('Age Distribution by Churn')
axes[0,0].set_xlabel('Age')
axes[0,0].set_ylabel('Count')
axes[0,0].legend()

# Gender by churn
gender_churn = pd.crosstab(df_clean['gender'], df_clean['churn'], normalize='index')
gender_churn.plot(kind='bar', ax=axes[0,1], color=['lightblue', 'lightcoral'])
axes[0,1].set_title('Churn Rate by Gender')
axes[0,1].set_ylabel('Churn Rate')
axes[0,1].tick_params(axis='x', rotation=0)

# Education by churn
education_churn = pd.crosstab(df_clean['education_level'], df_clean['churn'], normalize='index')
education_churn.plot(kind='bar', ax=axes[1,0], color=['lightblue', 'lightcoral'])
axes[1,0].set_title('Churn Rate by Education')
axes[1,0].set_ylabel('Churn Rate')
axes[1,0].tick_params(axis='x', rotation=45)

# Income distribution
axes[1,1].hist([df_clean[df_clean['churn']==0]['income'], df_clean[df_clean['churn']==1]['income']], 
               bins=20, alpha=0.7, label=['Retained', 'Churned'], color=['lightblue', 'lightcoral'])
axes[1,1].set_title('Income Distribution by Churn')
axes[1,1].set_xlabel('Income')
axes[1,1].set_ylabel('Count')
axes[1,1].legend()

plt.tight_layout()
plt.savefig('../outputs/02_demographic_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Financial behavior analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Monthly spending
axes[0,0].boxplot([df_clean[df_clean['churn']==0]['avg_monthly_spend'], 
                   df_clean[df_clean['churn']==1]['avg_monthly_spend']], 
                  labels=['Retained', 'Churned'])
axes[0,0].set_title('Monthly Spending by Churn')
axes[0,0].set_ylabel('Monthly Spend ($)')

# Credit utilization
axes[0,1].boxplot([df_clean[df_clean['churn']==0]['credit_utilization'], 
                   df_clean[df_clean['churn']==1]['credit_utilization']], 
                  labels=['Retained', 'Churned'])
axes[0,1].set_title('Credit Utilization by Churn')
axes[0,1].set_ylabel('Credit Utilization')

# Transaction count
axes[1,0].boxplot([df_clean[df_clean['churn']==0]['transaction_count'], 
                   df_clean[df_clean['churn']==1]['transaction_count']], 
                  labels=['Retained', 'Churned'])
axes[1,0].set_title('Transaction Count by Churn')
axes[1,0].set_ylabel('Transaction Count')

# Credit limit
axes[1,1].boxplot([df_clean[df_clean['churn']==0]['credit_limit'], 
                   df_clean[df_clean['churn']==1]['credit_limit']], 
                  labels=['Retained', 'Churned'])
axes[1,1].set_title('Credit Limit by Churn')
axes[1,1].set_ylabel('Credit Limit ($)')

plt.tight_layout()
plt.savefig('../outputs/03_financial_behavior_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation analysis with churn
numerical_cols = ['age', 'income', 'credit_limit', 'tenure_months', 'avg_monthly_spend', 
                 'credit_utilization', 'transaction_count', 'late_payment_count', 'risk_score']

correlations = df_clean[numerical_cols + ['churn']].corr()['churn'].sort_values(key=abs, ascending=False)

plt.figure(figsize=(10, 6))
correlations[1:].plot(kind='barh', color=['red' if x < 0 else 'blue' for x in correlations[1:]])
plt.title('Feature Correlations with Churn')
plt.xlabel('Correlation Coefficient')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/04_correlation_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("Top correlations with churn:")
print(correlations[1:6])

## EDA Key Insights

### Strongest Predictors of Churn:
1. **Tenure** (-0.18 correlation): Newer customers more likely to churn
2. **Income** (-0.07): Lower income customers show slightly higher churn
3. **Credit Limit** (-0.07): Lower credit limits associated with higher churn
4. **Risk Score** (+0.05): Higher risk scores correlate with churn
5. **Age** (+0.04): Older customers slightly more likely to churn

### Business Insights:
- Customer retention critical in first year of relationship
- High-value customers (higher income, credit limits) more loyal
- Middle-aged customers represent key churn segment
- Credit utilization and transaction patterns show clear differences
- Monthly spending significantly lower for churned customers

# Stage 4: Feature Engineering

## Creating new features to improve model performance

In [49]:
# Feature Engineering
df_features = df_clean.copy()

# Fix the tenure vs relationship duration inconsistency identified in data quality assessment
# Issue: correlation was 0.938 instead of 1.0, indicating inconsistent data
print("Fixing tenure/relationship duration inconsistency identified in Stage 1...")

# Use tenure_months as primary source (more precise) and recalculate relationship_duration_years
df_features['relationship_duration_years'] = df_features['tenure_months'] / 12

# Create new features
df_features['utilization_category'] = pd.cut(df_features['credit_utilization'], 
                                           bins=[0, 0.3, 0.7, 1.0], 
                                           labels=['Low', 'Medium', 'High'])

df_features['clv_proxy'] = df_features['avg_monthly_spend'] * df_features['tenure_months']

df_features['spend_to_limit_ratio'] = df_features['avg_monthly_spend'] / df_features['credit_limit']

df_features['engagement_score'] = (df_features['transaction_count'] * 
                                 df_features['avg_monthly_spend'] / 1000)

df_features['risk_adjusted_score'] = df_features['risk_score'] / df_features['credit_limit'] * 1000

df_features['age_group'] = pd.cut(df_features['age'], 
                                bins=[0, 35, 50, 65, 100], 
                                labels=['Young', 'Middle', 'Senior', 'Elder'])

new_features = ['utilization_category', 'clv_proxy', 'spend_to_limit_ratio', 
                'engagement_score', 'risk_adjusted_score', 'age_group']
print("New features created:", len(new_features))
print("Data consistency issue fixed")

Fixing tenure/relationship duration inconsistency identified in Stage 1...
New features created: 6
Data consistency issue fixed


In [50]:
# Verify the fix worked
print("Verification of tenure consistency fix:")
sample_check = df_features[['tenure_months', 'relationship_duration_years']].head()
print(sample_check)
print("New correlation:", df_features['tenure_months'].corr(df_features['relationship_duration_years']).round(6))
print("Data consistency issue resolved")

Verification of tenure consistency fix:
   tenure_months  relationship_duration_years
0             34                     2.833333
1             25                     2.083333
2             13                     1.083333
3            134                    11.166667
4             34                     2.833333
New correlation: 1.0
Data consistency issue resolved


In [51]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Check available columns
print("Available columns:", df_features.columns.tolist())

# Encode categorical variables (using correct column names)
label_encoders = {}
categorical_cols = ['gender', 'education_level', 'marital_status', 'income_category', 
                   'card_category', 'utilization_category', 'age_group']

for col in categorical_cols:
    if col in df_features.columns:
        le = LabelEncoder()
        df_features[col + '_encoded'] = le.fit_transform(df_features[col])
        label_encoders[col] = le

# Select features for modeling
feature_cols = ['age', 'tenure_months', 'transaction_count', 'late_payment_count',
                'avg_monthly_spend', 'credit_limit', 'credit_utilization', 'risk_score',
                'clv_proxy', 'spend_to_limit_ratio', 'engagement_score', 'risk_adjusted_score'] + \
               [col + '_encoded' for col in categorical_cols if col in df_features.columns]

X = df_features[feature_cols]
y = df_features['churn']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Training set:", X_train_scaled.shape)
print("Test set:", X_test_scaled.shape)
print("Churn rate in training:", round(y_train.mean(), 3))
print("Churn rate in test:", round(y_test.mean(), 3))

Available columns: ['customer_id', 'gender', 'age', 'education_level', 'marital_status', 'dependents', 'income', 'credit_limit', 'tenure_months', 'relationship_duration_years', 'avg_monthly_spend', 'monthly_balance', 'credit_utilization', 'transaction_count', 'cash_advance_count', 'card_category', 'late_payment_count', 'missed_payment_flag', 'last_payment_amount', 'online_services_used', 'reward_points', 'risk_score', 'churn', 'utilization_category', 'clv_proxy', 'spend_to_limit_ratio', 'engagement_score', 'risk_adjusted_score', 'age_group']
Training set: (8000, 18)
Test set: (2000, 18)
Churn rate in training: 0.111
Churn rate in test: 0.11


# Stage 5: Model Development

## Building predictive models to identify churn risk

## 5.1 Train Baseline Models

We start with two simple, interpretable models:
- **Logistic Regression** as a baseline.
- **Gradient Boosting** as a more flexible non-linear model.

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

# Logistic Regression
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)
y_prob_lr = lr_model.predict_proba(X_test_scaled)[:, 1]
lr_auc = roc_auc_score(y_test, y_prob_lr)

# Gradient Boosting
gb_model = GradientBoostingClassifier(
    n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42
)
gb_model.fit(X_train_scaled, y_train)
y_pred_gb = gb_model.predict(X_test_scaled)
y_prob_gb = gb_model.predict_proba(X_test_scaled)[:, 1]
gb_auc = roc_auc_score(y_test, y_prob_gb)

# Print results
print("Model Performance (ROC AUC):")
print(f"  Logistic Regression: {lr_auc:.3f}")
print(f"  Gradient Boosting:   {gb_auc:.3f}")
print(f"  Difference:          {gb_auc - lr_auc:.3f}")

Model Performance (ROC AUC):
  Logistic Regression: 0.713
  Gradient Boosting:   0.722
  Difference:          0.009


## 5.2 Evaluate Models

We compare classification performance for both models:
- **Classification reports** (precision, recall, F1) at the default 0.5 cutoff  
- **Confusion matrices** (heatmaps)  
- **ROC curve comparison** to visualize ranking quality

In [79]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

# 5.2.1 Classification Reports
print("Classification Report — Logistic Regression (0.5 threshold)")
print(classification_report(y_test, y_pred_lr, digits=3))

print("\nClassification Report — Gradient Boosting (0.5 threshold)")
print(classification_report(y_test, y_pred_gb, digits=3))

Classification Report — Logistic Regression (0.5 threshold)
              precision    recall  f1-score   support

           0      0.889     1.000     0.942      1779
           1      0.000     0.000     0.000       221

    accuracy                          0.889      2000
   macro avg      0.445     0.500     0.471      2000
weighted avg      0.791     0.889     0.837      2000


Classification Report — Gradient Boosting (0.5 threshold)
              precision    recall  f1-score   support

           0      0.892     0.990     0.938      1779
           1      0.280     0.032     0.057       221

    accuracy                          0.884      2000
   macro avg      0.586     0.511     0.498      2000
weighted avg      0.824     0.884     0.841      2000



In [ ]:
# 5.2.2 Confusion Matrices
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
sns.heatmap(confusion_matrix(y_test, y_pred_lr), annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Logistic Regression — Confusion Matrix')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

sns.heatmap(confusion_matrix(y_test, y_pred_gb), annot=True, fmt='d', cmap='Blues', ax=axes[1])
axes[1].set_title('Gradient Boosting — Confusion Matrix')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.savefig('../outputs/05_confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# 5.2.3 ROC Curve Comparison
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_prob_lr)
fpr_gb, tpr_gb, _ = roc_curve(y_test, y_prob_gb)

plt.figure(figsize=(6, 5))
plt.plot(fpr_lr, tpr_lr, label=f'LogReg (AUC = {lr_auc:.3f})')
plt.plot(fpr_gb, tpr_gb, label=f'GradBoost (AUC = {gb_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/06_roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

## 5.3 Derive Business Values from Data

We estimate the annual value of a customer using our dataset:
- **AVG_VALUE** = median monthly spend × 12  
We also set an assumption for the cost to run a retention campaign per targeted customer:
- **RETENTION_COST** (industry average)

In [84]:
# 5.3 Derive Business Values from Data

# Median monthly spend → annual value
AVG_VALUE = df_features['avg_monthly_spend'].median() * 12
print(f"Derived annual value per customer: ${AVG_VALUE:,.0f}")

# Retention cost assumption (industry estimate)
RETENTION_COST = 50.0
print(f"Retention cost per targeted customer: ${RETENTION_COST}")

Derived annual value per customer: $6,738
Retention cost per targeted customer: $50.0


## 5.4 Threshold Tuning for Profit

We optimize the decision cutoff for **business impact**, not just accuracy.  
For each possible threshold, we calculate:

Profit = (True Positives × Annual Value per Saved Customer)  
    − (True Positives + False Positives) × Retention Cost per Customer

Where:
- **True Positives (TP)** = actual churners correctly flagged
- **False Positives (FP)** = non-churners incorrectly targeted
- **Annual Value per Saved Customer** = from section 5.3
- **Retention Cost per Customer** = from section 5.3

### 5.4.1 Find profit-maximizing threshold
- Compute precision–recall over all thresholds
- Convert to counts (TP, FP) and calculate profit at each threshold
- Select the threshold that maximizes profit and evaluate metrics

### 5.4.2 Profit vs. Threshold Plot
- Visualize how expected profit changes across thresholds
- Highlight the chosen threshold

### 5.4.3 Confusion Matrix @ Best Threshold
- Show classification outcomes (TP, FP, TN, FN) at the selected operating point

In [86]:
from sklearn.metrics import precision_recall_curve, confusion_matrix, classification_report
import numpy as np

# Calculate precision-recall curve
precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob_gb)

# Positive class count (number of churners)
P = y_test.sum()

# For each threshold, get predicted positives, true positives, false positives
pred_pos = np.array([(y_prob_gb >= t).sum() for t in thresholds])
TP = recalls[:-1] * P  # skip last recall element to align with thresholds
FP = pred_pos - TP

# Profit calculation
profits = TP * AVG_VALUE - (TP + FP) * RETENTION_COST

# Find best threshold
best_idx = np.argmax(profits)
best_threshold = thresholds[best_idx]

print(f"Best threshold for maximum profit: {best_threshold:.3f}")
print(f"Expected profit at best threshold: ${profits[best_idx]:,.0f}")

# Predictions at best threshold
y_pred_gb_best = (y_prob_gb >= best_threshold).astype(int)

# Metrics
cm_best = confusion_matrix(y_test, y_pred_gb_best)
precision_best = cm_best[1,1] / (cm_best[1,1] + cm_best[0,1]) if (cm_best[1,1] + cm_best[0,1]) > 0 else 0
recall_best = cm_best[1,1] / (cm_best[1,1] + cm_best[1,0]) if (cm_best[1,1] + cm_best[1,0]) > 0 else 0

print("\nMetrics @ best threshold")
print("Confusion Matrix:\n", cm_best)
print(f"Precision: {precision_best:.3f} | Recall: {recall_best:.3f}")
print("\nClassification Report @ best threshold:")
print(classification_report(y_test, y_pred_gb_best, digits=3))

Best threshold for maximum profit: 0.016
Expected profit at best threshold: $1,394,710

Metrics @ best threshold
Confusion Matrix:
 [[ 246 1533]
 [   1  220]]
Precision: 0.125 | Recall: 0.995

Classification Report @ best threshold:
              precision    recall  f1-score   support

           0      0.996     0.138     0.243      1779
           1      0.125     0.995     0.223       221

    accuracy                          0.233      2000
   macro avg      0.561     0.567     0.233      2000
weighted avg      0.900     0.233     0.241      2000



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(thresholds, profits, label='Profit', linewidth=2)
plt.axvline(best_threshold, linestyle='--', color='red', label=f'Best Threshold ({best_threshold:.3f})')
plt.title("Profit vs. Decision Threshold", fontsize=14, fontweight='bold')
plt.xlabel("Threshold")
plt.ylabel("Profit ($)")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/07_profit_vs_threshold.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(5,4))
sns.heatmap(cm_best, annot=True, fmt='d', cmap='Blues')
plt.title(f'GB Confusion Matrix @ Threshold={best_threshold:.3f}')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.savefig('../outputs/08_optimal_threshold_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## 5.5 ROI Analysis at Best Threshold

We translate the model results into **real business impact** using the optimal decision threshold from Section 5.4.

Definitions:
- **True Positives (TP)** → correctly predicted churners (saved customers)
- **False Positives (FP)** → non-churners incorrectly targeted
- **Annual Value per Saved Customer** → from Section 5.3
- **Retention Cost per Customer** → from Section 5.3
- **Acquisition Cost** → cost to replace a lost customer

We calculate:
1. **Customers Saved** = TP × (Retention Success Rate)
2. **Revenue Saved** = Customers Saved × Annual Value
3. **Acquisition Cost Avoided** = Customers Saved × Acquisition Cost
4. **Campaign Cost** = (TP + FP) × Retention Cost
5. **Net ROI** = (Revenue Saved + Acquisition Cost Avoided) − Campaign Cost
6. **ROI Ratio** = (Total Benefits) ÷ Campaign Cost

In [ ]:
# Business constants 
ACQ_COST = 250.0  # Acquisition cost to replace a lost customer
SUCCESS_RATE = 0.6  # Retention campaign success rate

# Confusion matrix values from best threshold
TP_best = cm_best[1,1]
FP_best = cm_best[0,1]

# Customers saved after campaign success rate
customers_saved = TP_best * SUCCESS_RATE

# Financial impact
revenue_saved = customers_saved * AVG_VALUE
acquisition_cost_avoided = customers_saved * ACQ_COST
campaign_cost = (TP_best + FP_best) * RETENTION_COST

total_benefits = revenue_saved + acquisition_cost_avoided
net_roi = total_benefits - campaign_cost
roi_ratio = total_benefits / campaign_cost if campaign_cost > 0 else 0

# Print results
print("="*60)
print("ROI ANALYSIS @ BEST THRESHOLD")
print("="*60)
print(f"True Positives (TP): {TP_best}")
print(f"False Positives (FP): {FP_best}")
print(f"Customers saved (after success rate): {customers_saved:.0f}")
print(f"Revenue saved: ${revenue_saved:,.0f}")
print(f"Acquisition cost avoided: ${acquisition_cost_avoided:,.0f}")
print(f"Total benefits: ${total_benefits:,.0f}")
print(f"Campaign cost: ${campaign_cost:,.0f}")
print(f"Net ROI: ${net_roi:,.0f}")
print(f"ROI ratio: {roi_ratio:.2f}x")

ROI ANALYSIS @ BEST THRESHOLD
True Positives (TP): 220
False Positives (FP): 1533
Customers saved (after success rate): 132
Revenue saved: $889,416
Acquisition cost avoided: $33,000
Total benefits: $922,416
Campaign cost: $87,650
Net ROI: $834,766
ROI ratio: 10.52x


In [ ]:
# Simple ROI bar chart
import matplotlib.pyplot as plt

categories = ["Revenue Saved", "Acq. Cost Avoided", "Campaign Cost", "Net ROI"]
values = [revenue_saved, acquisition_cost_avoided, -campaign_cost, net_roi]
colors = ['#4CAF50', '#2196F3', '#F44336', '#9C27B0']

plt.figure(figsize=(8,5))
bars = plt.bar(categories, values, color=colors)
plt.title("ROI Breakdown @ Best Threshold", fontsize=14, fontweight='bold')
plt.ylabel("Amount ($)")

# Add labels
for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width()/2, value + (max(values)*0.02),
             f"${abs(value):,.0f}", ha='center', fontweight='bold')

plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/09_roi_breakdown.png', dpi=300, bbox_inches='tight')
plt.show()

## 5.6 Sensitivity Analysis

Goal: show how ROI changes if our business assumptions change.

We vary:
- **Retention Cost** per targeted customer
- **Retention Success Rate** on true churners
- (Optional) **Include/Exclude Acquisition Cost Avoided**

We also compute **break-even points**:
- *Break-even retention cost* — the highest we can spend per customer and still break even
- *Break-even success rate* — the minimum success rate needed to break even at a given cost

In [92]:
# 5.6.1 Break-even math at our chosen operating point

import numpy as np

# Pull TP/FP from the confusion matrix at best threshold
TP_best = cm_best[1,1]
FP_best = cm_best[0,1]

# Toggle whether to include acquisition cost avoided in benefits
INCLUDE_ACQ = True

value_per_save = AVG_VALUE + (ACQ_COST if INCLUDE_ACQ else 0.0)

# Break-even retention cost (given success rate)
def breakeven_retention_cost(success_rate):
    # Net ROI = TP*s*value - (TP+FP)*RC = 0  =>  RC* = (TP*s*value)/(TP+FP)
    denom = (TP_best + FP_best)
    return (TP_best * success_rate * value_per_save) / denom if denom > 0 else np.nan

# Break-even success rate (given retention cost)
def breakeven_success_rate(retention_cost):
    # 0 = TP*s*value - (TP+FP)*RC  =>  s* = ((TP+FP)*RC)/(TP*value)
    denom = (TP_best * value_per_save)
    return ((TP_best + FP_best) * retention_cost) / denom if denom > 0 else np.nan

# Examples around your current assumptions
RC_current = RETENTION_COST
SR_current = 0.60  # from 5.5

rc_star = breakeven_retention_cost(SR_current)
sr_star = breakeven_success_rate(RC_current)

print("=== Break-even checks (at best threshold) ===")
print(f"Current assumptions: RETENTION_COST=${RC_current:.0f}, SUCCESS_RATE={SR_current:.2f}, "
      f"Include ACQ={INCLUDE_ACQ}")
print(f"Break-even retention cost (given success={SR_current:.2f}): ${rc_star:,.2f}")
print(f"Break-even success rate (given cost=${RC_current:.0f}): {sr_star:.3f}")

=== Break-even checks (at best threshold) ===
Current assumptions: RETENTION_COST=$50, SUCCESS_RATE=0.60, Include ACQ=True
Break-even retention cost (given success=0.60): $526.19
Break-even success rate (given cost=$50): 0.057


In [ ]:
# 5.6.2 Sensitivity grid over Retention Cost and Success Rate

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

TP_best = cm_best[1,1]
FP_best = cm_best[0,1]
value_per_save = AVG_VALUE + (ACQ_COST if INCLUDE_ACQ else 0.0)

retention_grid = [20, 35, 50, 75, 100, 150]
success_grid   = [0.30, 0.45, 0.60, 0.75, 0.90]

rows = []
for rc in retention_grid:
    for sr in success_grid:
        customers_saved = TP_best * sr
        benefits = customers_saved * value_per_save
        cost = (TP_best + FP_best) * rc
        net = benefits - cost
        roi_ratio = (benefits / cost) if cost > 0 else np.nan
        rows.append({"Retention_Cost": rc, "Success_Rate": sr, "Net_ROI": net, "ROI_Multiple": roi_ratio})

sens_df = pd.DataFrame(rows)

# Show top 10 scenarios by Net ROI
print("Top scenarios by Net ROI:")
display(sens_df.sort_values("Net_ROI", ascending=False).head(10))

# Heatmap of Net ROI
heatmap_df = sens_df.pivot(index="Success_Rate", columns="Retention_Cost", values="Net_ROI").sort_index(ascending=True)

plt.figure(figsize=(8,5))
sns.heatmap(heatmap_df, annot=True, fmt=".0f", cmap="YlGnBu")
plt.title(f"Net ROI Sensitivity (Include ACQ={INCLUDE_ACQ})")
plt.ylabel("Success Rate")
plt.xlabel("Retention Cost ($)")
plt.tight_layout()
plt.savefig('../outputs/10_sensitivity_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

In [94]:
# 5.6.3 Quick one-liners to quote in the write-up

print("=== Quick quotes ===")
print(f"- With success rate 60% and cost ${RETENTION_COST:.0f}, ROI multiple is "
      f"{(TP_best*0.60*value_per_save)/((TP_best+FP_best)*RETENTION_COST):.2f}x.")
print(f"- Break-even retention cost at 60% success is ${breakeven_retention_cost(0.60):.2f}.")
print(f"- Break-even success rate at ${RETENTION_COST:.0f} cost is {breakeven_success_rate(RETENTION_COST):.2%}.")

=== Quick quotes ===
- With success rate 60% and cost $50, ROI multiple is 10.52x.
- Break-even retention cost at 60% success is $526.19.
- Break-even success rate at $50 cost is 5.70%.


## Stage 5 Summary — Model Development & Business Impact

**Model Comparison (ROC AUC)**  
- Logistic Regression: 0.713  
- Gradient Boosting: 0.722 (slightly better performance)

**Best Threshold for Profit (Gradient Boosting)**  
- Optimal cutoff: **0.016**  
- Expected profit: **$1.39M/year**  
- Recall = 0.995 (captures almost all churners), Precision = 0.125 (many non-churners targeted)  
- Low precision is acceptable here due to low retention cost per customer.

**ROI Analysis @ Best Threshold**  
- Customers saved (after 60% success rate): 132  
- Revenue saved: **$889K**  
- Acquisition cost avoided: **$33K**  
- Campaign cost: **$87.6K**  
- **Net ROI: $834.8K** → ROI ratio = **10.52×**  

**Sensitivity Insights**  
- ROI remains positive even if retention cost rises to ~$500 per customer (at 60% success rate).  
- Break-even success rate at $50 cost is only ~5.7%, meaning the campaign is highly resilient to underperformance.  
- Higher success rates or lower costs increase profitability substantially.

**Business Takeaways**  
- The model enables **high-coverage targeting** for churn prevention at minimal financial risk.  
- Even with low precision, cheap intervention costs make the strategy worthwhile.  
- Key risk: unnecessary targeting of non-churners — could be improved with further precision-focused model tuning.

# Stage 6: Strategic Insights & Recommendations

## Key Insights

1. **Model Performance**
   - Gradient Boosting achieved the highest ROC AUC score (0.722), slightly outperforming Logistic Regression (0.713).  
   - While the performance gap is modest, Gradient Boosting’s ability to better rank customers by churn probability makes it the preferred choice for operational use.

2. **Threshold Tuning & Business Alignment**
   - A profit-optimized threshold of **0.016** was identified, based on maximizing expected net ROI.
   - At this threshold, recall is extremely high (0.995), meaning almost all true churners are captured.
   - Precision is low (0.125), so many non-churners will also be targeted — but given the low $50 intervention cost, this is financially acceptable.

3. **Financial Resilience**
   - Current assumptions yield a **Net ROI of $834,766** and an ROI multiple of **10.52×**.
   - Break-even analysis shows the strategy remains profitable even if retention cost per customer increases to ~$526 at a 60% success rate.
   - Minimum success rate to break even at $50 cost is just **5.7%**, indicating strong resilience.

4. **Drivers of Profitability**
   - The main profitability driver is the **low cost of intervention** relative to the high annual value per retained customer (~$6,738).
   - Even with a high false-positive rate, the cost-to-benefit ratio remains strongly favorable.

---

## Strategic Recommendations

1. **Deploy the Gradient Boosting Model**
   - Implement Gradient Boosting with a threshold of 0.016 for an initial churn prevention campaign.
   - Ensure probability calibration is monitored to avoid drift over time.

2. **Improve Campaign Success Rate**
   - Focus marketing and retention team efforts on improving success from 60% toward 75%+.
   - Even without improving precision, a higher success rate significantly increases ROI.

3. **Segment Targeted Customers**
   - Apply a secondary segmentation layer to predicted churners (e.g., by CLV, tenure, or product usage).
   - Prioritize high-value or at-risk segments to maximize ROI while controlling unnecessary spend.

4. **Monitor and Recalibrate**
   - Track precision, recall, and ROI after each campaign cycle.
   - Adjust thresholds dynamically if intervention costs change or if customer churn patterns shift.

5. **Test Precision-Boosting Strategies**
   - Explore ensemble models or post-prediction filtering to reduce false positives without significantly reducing recall.
   - Consider dual-threshold strategies to create “high-priority” and “medium-priority” intervention tiers.

---

## Next Steps

- **Data Enrichment:** Add additional behavioral variables (e.g., customer service interactions, payment history, product engagement) to improve model discrimination.
- **Pilot & Measure:** Run a controlled A/B pilot comparing the targeted group against a control group to validate modeled ROI with real-world results.
- **Sensitivity Updates:** Update cost and revenue assumptions quarterly to ensure financial models remain relevant.
- **Operational Integration:** Work with CRM and marketing teams to automate targeting workflows and track post-intervention churn outcomes.

---

> **Bottom Line:**  
> The Gradient Boosting model with a profit-tuned threshold offers a high-recall, low-cost strategy for churn prevention that is financially robust under a wide range of conditions. While the current approach favors recall over precision, the economic upside justifies broad targeting. Continued monitoring, segmentation, and success rate improvements will further enhance ROI.